<a href="https://colab.research.google.com/github/meng430/repo0/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():
    ngrok.set_auth_token(userdata.get('ngrok_auth_token'))
    public_url = ngrok.connect(5000)
    print("Ngrok Public URL:", public_url)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage,
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('Line_Channel_access_token'))
handler = WebhookHandler(userdata.get('Line_Channel_secret'))

@app.route("/callback", methods=['POST'])

def callback():
  signature = request.headers['X-Line-Signature']
  body = request.get_data(as_text=True)
  try:
      handler.handle(body, signature)
  except InvalidSignatureError:
      abort(400)
  return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash-exp")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
  user_msg = event.message.text
  response = ask_gemini(user_msg)
  with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'sticker':
          #回覆貼圖
          reply = StickerMessage(
            package_id='446',
            sticker_id='1989'
          )
        elif action == 'image':
          #回覆圖片
          reply = ImageMessage(
              original_content_url='https://cdn.pixabay.com/photo/2017/06/16/07/12/taipei-101-2408050_1280.jpg',
              preview_image_url='https://cdn.pixabay.com/photo/2017/06/16/07/12/taipei-101-2408050_1280.jpg'
          )
        elif action == 'video':
          #回覆影片
          reply = VideoMessage(
            original_content_url='https://cdn.pixabay.com/video/2025/01/15/252672_large.mp4',
            preview_image_url='https://cdn.pixabay.com/video/2025/01/15/252672_large.jpg'
          )
        elif action == 'location':
          reply = LocationMessage(
            title='Taipei 101',
            address='110台北市信義區信義路五段7號',
            latitude=25.034199559170606,
            longitude=121.56456035396677
          )
        else:
          response = ask_gemini(action)
          reply = TextMessage(text=response)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

Ngrok Public URL: NgrokTunnel: "https://0af2-34-74-236-12.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/May/2025 10:41:05] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/May/2025 10:41:22] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/May/2025 10:41:31] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/May/2025 10:41:45] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/May/2025 10:42:00] "POST /callback HTTP/1.1" 200 -
ERROR:__main__:Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exc